<a href="https://colab.research.google.com/github/hiya906/my-machine-learning/blob/master/movie_review_LSTM_CNN_co_teaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDS NLP Challenge

- 이 Challenge는 자연어 처리를 통해 영화 감상평을 보고 영화 평점을 예측하는 것을 목적으로 합니다.
- 데이터는 2 종류가 있습니다: IMDB (영어), NSMC (한국어)
- 전처리, 학습 및 평가 템플릿은 아래와 같이 제공됩니다. 원하시는 대로 Parameter를 조절하고 'MyModel'에 해당하는 본인의 모델을 구현하여 실험하시면 됩니다.
- Baseline 성능은 XXX와 같습니다.
- Challenge 설명 PPT에 더 자세한 설명이 들어있습니다.

In [0]:
import os
os.mkdir('data')
os.mkdir('models')
os.mkdir('util')

In [2]:
import tensorflow as tf
print(tf.__version__)
print(dir(tf.feature_column))
# !pip install tensorflow==1.4.0

1.4.0
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_allowed_symbols', 'bucketized_column', 'categorical_column_with_hash_bucket', 'categorical_column_with_identity', 'categorical_column_with_vocabulary_file', 'categorical_column_with_vocabulary_list', 'crossed_column', 'embedding_column', 'indicator_column', 'input_layer', 'linear_model', 'make_parse_example_spec', 'numeric_column', 'weighted_categorical_column']


In [0]:
# IMPORT PACKAGE
import time
import argparse
import numpy as np
import tensorflow as tf

from util.Data_loader import load_data
from util.Dataset import Dataset
from util.Parser import vocab_dictionary, parse_data

from models.Baseline_LSTM import LSTM
from models.Baseline_Char_CNN import Char_CNN

In [0]:
# imdb, nsmc
DATA_NAME = 'imdb'

# 전처리 방법
# imdb (영어) = word 
# nsmc (한글) = eumjeol or char (음절 or 음소)
PARSER = 'word'

MAXLEN = 300
# BATCH_SIZE = 256
# LEARNING_RATE = 0.01
BATCH_SIZE = 256
LEARNING_RATE = 0.01

In [0]:
train_file = DATA_NAME + '_train.txt'
test_file = DATA_NAME + '_test.txt'
train_comments, train_ratings, test_comments, test_ratings = load_data('data', train_file, test_file)

str2idx = vocab_dictionary(train_comments, PARSER)
idx2str = {i: s for s, i in str2idx.items()}

CHARSIZE = len(str2idx)

In [0]:
# 전처리
train_x = parse_data(train_comments, str2idx, PARSER)
train_y = train_ratings

test_x = parse_data(test_comments, str2idx, PARSER)
test_y = test_ratings

In [0]:
# 데이터셋
train_dataset = Dataset(train_x, train_y, MAXLEN, BATCH_SIZE, shuffle=True)
test_dataset = Dataset(test_x, test_y, MAXLEN, BATCH_SIZE, shuffle=False)

In [0]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np


class My_LSTM():
    def __init__(self, x, y, in_len, embedding, rnn_hidden, bi, fc_layers, maxlen, char_size, lr):
        self.embedding = embedding
        self.rnn_hidden = rnn_hidden
        self.bi = bi
        self.fc_layers = fc_layers
        self.input_len = maxlen
        self.char_size = char_size
        self.lr = lr
        self.squered_error = None

        with tf.name_scope("Input-Layer"):
            # Input
#             self.x = tf.placeholder(tf.int64, shape=[None, self.input_len], name="input_x")
#             self.y = tf.placeholder(tf.float32, shape=[None], name="output_y")
            self.x = x
            self.y = y
#             self.x_len = tf.placeholder(tf.int64, shape=[None], name="input_len")
            self.x_len = in_len

            embedding_matrix = tf.Variable(tf.random_normal([self.char_size, self.embedding], stddev=0.01),
                                           name='Embedding_matrix')

        # EMBEDDING LAYERS
        with tf.name_scope("Embedding-Layer"):
            # CNN (tf.conv2d) : input = [batch, in_height, in_width, in_channels]
            # NLP embedding only has 1 channels
            # shape = (Batch, input_len, Embedding, 1)
            x_emb = tf.nn.embedding_lookup(embedding_matrix, self.x)

        input_shape = tf.shape(x_emb)

        # LSTM Layer
        with tf.name_scope('RNN-Layer'):
            cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.rnn_hidden)
            init_state_fw = cell.zero_state(batch_size=input_shape[0], dtype=tf.float32)
            if self.bi:
                cell2 = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.rnn_hidden)
                init_state_bw = cell.zero_state(batch_size=input_shape[0], dtype=tf.float32)

                output, final_state = tf.nn.bidirectional_dynamic_rnn(cell, cell2, x_emb,
                                                                      initial_state_fw=init_state_fw,
                                                                      initial_state_bw=init_state_bw, time_major=False,
                                                                      sequence_length=self.x_len)
                h = tf.concat([final_state[0].h, final_state[1].h], 1)
                output = tf.concat(output, 2)
            else:
                output, final_state = tf.nn.dynamic_rnn(cell, x_emb, initial_state=init_state_fw, time_major=False,
                                                        sequence_length=self.x_len)
                h = final_state.h

            output_shape = output.get_shape()

            d = output_shape[-1].value

            # 1. Use Last Hidden Only  (Batch x Hidden)
            # rnn_output = h

            # 2. Use Mean Pooling      (Batch x Hidden)
            rnn_output = tf.reduce_mean(output, axis=1)

            # 3. Use All Hidden      (Batch x d)
            # d = output_shape[1].value * output_shape[2].value
            # rnn_output = tf.reshape(output, [-1, d])

        fc_input = rnn_output
        for i, fc_dim in enumerate(self.fc_layers):
            with tf.name_scope("FC-Layer-" + str(i)):
                W = tf.Variable(tf.random_normal([d, fc_dim], stddev=0.01), name="FC_W" + str(i))
                b = tf.Variable(tf.random_normal([fc_dim], stddev=0.01), name="FC_b" + str(i))

                fc_input = tf.nn.xw_plus_b(fc_input, W, b, name="FC" + str(i))
                fc_input = tf.nn.dropout(tf.nn.relu(fc_input), 0.5)
                d = fc_dim

        fc_output = fc_input

        with tf.name_scope("Output-Layer"):

            W = tf.Variable(tf.random_normal([d, 1], stddev=0.01), name="Output_W")
            b = tf.Variable(tf.random_normal([1], stddev=0.01), name="Output_b")

            output = tf.squeeze(tf.matmul(fc_output, W) + b, 1)
            self.pred = output

        with tf.name_scope("Loss"):
            self.squared_error = tf.square(self.y - output)
            self.loss = tf.reduce_mean(self.squared_error)

        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        self.train_step = optimizer.minimize(self.loss)

    def __str__(self):
        name = "LSTM"
        bi = "Bidirectional : " + str(self.bi)
        embeddings = "Embedding Size : " + str(self.embedding)
        hidden = "RNN HIDDEN : " + str(self.rnn_hidden)
        layers = "FC Layers : " + str(self.fc_layers)

        return '\n'.join([name, bi, embeddings, hidden, layers])

In [0]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np

class MyModel_CNN():
    def __init__(self, x, y, embedding, conv_layers, fc_layers, maxlen, char_size, lr):
        self.embedding = embedding
        self.input_len = maxlen
        self.char_size = char_size
        self.conv_layers = conv_layers
        self.fc_layers = fc_layers
        self.lr = lr
        self.squared_error = None
        
#         self.output_keep_prob = tf.placeholder(tf.float32, name='output_keep_prob')

        with tf.name_scope("Input-Layer"):
            # Input
#             self.x = tf.placeholder(tf.int64, shape=[None, self.input_len], name="input_x")
#             self.y = tf.placeholder(tf.float32, shape=[None], name="output_y")
            self.x = x
            self.y = y
            embedding_matrix = tf.Variable(tf.random_normal([self.char_size, self.embedding], stddev=0.01), name='Embedding_matrix')
            print("Embedding Matrix: ", embedding_matrix.shape)

        # EMBEDDING LAYERS
        with tf.name_scope("Embedding-Layer"):
            cnn_x = tf.nn.embedding_lookup(embedding_matrix, self.x)
            cnn_x = tf.expand_dims(cnn_x, -1)
        
        
        # ================================ Version 2 =================================
        # CONVOLUTION LAYERS
        for i, conv_info in enumerate(self.conv_layers):
            print("CNN Input: ", cnn_x.shape)
            with tf.name_scope("Conv-Layer" + str(i)):
                filter_width = cnn_x.get_shape()[2].value
                filter_shape = [conv_info[1], filter_width, 1, conv_info[0]]

                W = tf.Variable(tf.random_normal(filter_shape,  mean=0.0, stddev=0.01), dtype=tf.float32,
                                name='Conv_W')  # large = 0.02 , small = 0.05
                b = tf.Variable(tf.random_normal(shape=[conv_info[0]],  mean=0.0, stddev=0.01), dtype=tf.float32,
                                name='Conv_b')

                conv = tf.nn.conv2d(cnn_x, W, [1, 1, 1, 1], "VALID", name="conv")
                cnn_x = tf.nn.bias_add(conv, b)

            with tf.name_scope("Non-Linear"):
                cnn_x = tf.nn.relu(cnn_x)
#                 cnn_x = tf.nn.tanh(cnn_x)
            print("CNN Output: ", cnn_x.shape)
            if conv_info[-1] != -1:
                print("Pooling Input: ", cnn_x.shape)
                with tf.name_scope("Max-Polling"):
                    pool_shape = [1, conv_info[-1], 1, 1]
                    pool = tf.nn.max_pool(cnn_x, ksize=pool_shape, strides=pool_shape, padding="VALID")
                    cnn_x = tf.transpose(pool, [0, 1, 3, 2])
                print("Pooling Output: ", cnn_x.shape)
            else:
                cnn_x = tf.transpose(cnn_x, [0, 1, 3, 2])
        cnn_output = tf.squeeze(cnn_x, axis=3)

        # Flatten cnn_output: (batch, height, width)  --> (batch, height * width)
        out_shape = cnn_output.get_shape()
        d = out_shape[1].value * out_shape[2].value
        print('out_shape[1].value: ', out_shape[1].value)
        print('out_shape[2].value: ', out_shape[2].value)
        fc_input = tf.reshape(cnn_output, [-1, d])
        print('fc_input: ', fc_input)

        # Add dropout
#         keep_prob = tf.placeholder(tf.float32)
#         h_drop = tf.nn.dropout(fc_input, self.output_keep_prob)
        
        # Regression layer
        print("Output Layer input: ", fc_input.shape)
        with tf.name_scope("Output-Layer"):
            W = tf.Variable(tf.random_normal([d, 1], stddev=0.01), name="Output_W")
            b = tf.Variable(tf.random_normal([1], stddev=0.01), name="Output_b")

            output = tf.squeeze(tf.matmul(fc_input, W) + b, 1)
            self.pred = output
        print("Output Layer output: ", output.shape)

        with tf.name_scope("Loss"):
            self.squared_error = tf.square(self.y - output)
            self.loss = tf.reduce_mean(self.squared_error)

        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        self.train_step = optimizer.minimize(self.loss)

In [0]:
tf.reset_default_graph()

In [0]:
class MyModel:
    def __init__(self, embedding, conv_layers, fc_layers, maxlen, char_size, lr):
        # Input Placeholders
        self.embedding = embedding
        self.input_len = maxlen
        self.char_size = char_size
        self.conv_layers = conv_layers
        self.fc_layers = fc_layers
        self.lr = lr   
                 
        self.x = tf.placeholder(tf.int64, shape=[None, self.input_len], name="input_x")
        self.y = tf.placeholder(tf.float32, shape=[None], name="output_y")
        self.x_len = tf.placeholder(tf.int64, shape=[None], name="input_len")

        self.idx_sorted_1 = tf.placeholder(tf.int64, shape=[None], name="idx_sorted_1")
        self.idx_sorted_2 = tf.placeholder(tf.int64, shape=[None], name="idx_sorted_2")

        print('====================== Model 1 ======================')
        self.model1 = My_LSTM(self.x, self.y, self.x_len, 32, 32, False, [], MAXLEN, CHARSIZE, 0.01)
#         self.x_len1 = self.model1.x_len
#         print('X_len 1 : ', self.x_len1)
        print('====================== Model 2 ======================')
        self.model2 = MyModel_CNN(self.x, self.y, 32, [[16, 3, -1], [16, 3, 3]], [], MAXLEN, CHARSIZE, 0.01)
        
        self.loss1 = self.model1.squared_error
        self.loss2 = self.model2.squared_error

        print('TMP Loss 1 : ', self.loss1)
        print('TMP Loss 2 : ', self.loss2)            
        
        # Model Network....        
        
        self.pred_1 = self.model1.pred
        self.pred_2 = self.model2.pred
        
        # Define loss        
        self.train_loss1 = tf.reduce_mean(tf.gather(self.loss1, self.idx_sorted_2))
        self.train_loss2 = tf.reduce_mean(tf.gather(self.loss2, self.idx_sorted_1))
        print('Train Loss 1 : ', self.train_loss1)
        print('Train Loss 2 : ', self.train_loss2)
        
        # train_step = "optimize operation"
        optimizer_1 = tf.train.AdamOptimizer(learning_rate=self.lr)
        optimizer_2 = tf.train.AdamOptimizer(learning_rate=self.lr)
        self.train_step_1 = optimizer_1.minimize(self.train_loss1)
        self.train_step_2 = optimizer_2.minimize(self.train_loss2)

In [22]:
MODEL_NAME = 'MY_MODEL2'

if MODEL_NAME == 'LSTM':
    model = LSTM(32, 32, False, [], MAXLEN, CHARSIZE, 0.01)
    input_x = model.x
    output_y = model.y
    input_len = model.x_len
    pred = model.pred
    loss = model.loss
    train_op = model.train_step
elif MODEL_NAME == 'CNN':
    model = Char_CNN(32, [[32, 3, -1], [32, 3, 3]], [], MAXLEN, CHARSIZE, 0.01)
    input_x = model.x
    output_y = model.y
    pred = model.pred
    loss = model.loss
    train_op = model.train_step
elif MODEL_NAME == 'MY_MODEL':
    model = MyModel(32, [[16, 3, -1], [16, 3, 3]], [], MAXLEN, CHARSIZE, 0.0001)
    input_x = model.x
    output_y = model.y
    pred = model.pred
    loss = model.loss
    train_op = model.train_step
    output_keep_prob = model.output_keep_prob
    
elif MODEL_NAME == 'MY_MODEL2':
    model= MyModel(32, [[16, 3, -1], [16, 3, 3]], [], MAXLEN, CHARSIZE, 0.0001) # 위에서 새롭게 정의한 것을 가져오기
    input_x = model.x
    output_y = model.y    

    pred_1 = model.pred_1
    loss_1 = model.loss1
    train_loss_1 = model.train_loss1
    idx_1 = model.idx_sorted_1
    train_op_1 = model.train_step_1
    input_len = model.x_len

    pred_2 = model.pred_2
    loss_2 = model.loss2
    train_loss_2 = model.train_loss2
    idx_2 = model.idx_sorted_2
    train_op_2 = model.train_step_2
    
#     output_keep_prob = model.output_keep_prob
else:
    raise NotImplementedError

====================== Model 1 ======================
====================== Model 2 ======================
Embedding Matrix:  (340183, 32)
CNN Input:  (?, 300, 32, 1)
CNN Output:  (?, 298, 1, 16)
CNN Input:  (?, 298, 16, 1)
CNN Output:  (?, 296, 1, 16)
Pooling Input:  (?, 296, 1, 16)
Pooling Output:  (?, 98, 16, 1)
out_shape[1].value:  98
out_shape[2].value:  16
fc_input:  Tensor("Reshape:0", shape=(?, 1568), dtype=float32)
Output Layer input:  (?, 1568)
Output Layer output:  (?,)
TMP Loss 1 :  Tensor("Loss/Square:0", shape=(?,), dtype=float32)
TMP Loss 2 :  Tensor("Loss_1/Square:0", shape=(?,), dtype=float32)
Train Loss 1 :  Tensor("Mean:0", shape=(), dtype=float32)
Train Loss 2 :  Tensor("Mean_1:0", shape=(), dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
# 학습 시작하기
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [25]:
## LSTM RUNNING ###

best_rmse = 1000000
best_epoch = -1

EPOCHs = 100
DISPLAY_STEP = 50

# TRAINING LOOP
for epoch in range(1, EPOCHs + 1):
    epoch_loss1 = 0.0
    epoch_loss2 = 0.0
    epoch_start = time.time()
    # TRAIN EACH BATCH
    for i, (train_x, train_y) in enumerate(train_dataset):
        # PAD BATCH DATA
        train_x_len = [len(x) for x in train_x]
        train_x_pad = np.ones((len(train_x), MAXLEN), dtype=np.int32) * 2  # PAD : 2
        for idx, s in enumerate(train_x):
            length = len(s)
            if length < MAXLEN:
                train_x_pad[idx, :length] = np.array(s)
            else:
                train_x_pad[idx:, :MAXLEN] = np.array(s)[:MAXLEN]  # Truncate from the front
        
        # DEFINE feed_dict
        # ex) feed_dict = {input_x: train_x_pad, output_y: train_y}
        # feed_dict = {}
        # feed_dict = {input_x: train_x_pad, output_y: train_y}
        feed_dict = {input_x: train_x_pad, output_y: train_y, input_len: train_x_len}
        tmp_loss_1, tmp_loss_2 = sess.run([loss_1, loss_2], feed_dict=feed_dict)

        idx_sorted_1 = np.argsort(tmp_loss_1)[:100]
        idx_sorted_2 = np.argsort(tmp_loss_2)[:100]
        feed_dict = {input_x: train_x_pad, output_y: train_y, idx_1: idx_sorted_1, idx_2: idx_sorted_2, input_len: train_x_len}
        
        t = time.time()
        # train_op: train, optimize
        # loss : batch loss
        _, l1 = sess.run([train_op_1, train_loss_1], feed_dict=feed_dict)
        _, l2 = sess.run([train_op_2, train_loss_2], feed_dict=feed_dict)
        elapsed = time.time() - t

        epoch_loss1 += l1
        epoch_loss2 += l2
        
        # PRINT LOSS
        if (i + 1) % DISPLAY_STEP == 0:
#             print("111111: ", train_dataset.num_batch, l1, elapsed)
            print('[%3d/%3d] loss1 = %.4f, time elapsed = %.2f' % (i + 1, train_dataset.num_batch, l1, elapsed))
            print('[%3d/%3d] loss2 = %.4f, time elapsed = %.2f' % (i + 1, train_dataset.num_batch, l2, elapsed))
    epoch_end = time.time() - epoch_start
    print('Epoch %3d >> Epoch loss1: %.4f , time elapsed %.4f' % (epoch, epoch_loss1, epoch_end))
    print('Epoch %3d >> Epoch loss2: %.4f , time elapsed %.4f' % (epoch, epoch_loss2, epoch_end))
    
    # EVALUATE ON TEST DATA 
    se_1 = 0
    se_2 = 0
    total_1 = 0
    total_2 = 0
    for i, (test_x, test_y) in enumerate(test_dataset):
        # PAD
        test_x_len = [len(x) for x in test_x]
        test_x_pad = np.ones((len(test_x), MAXLEN), dtype=np.int32) * 2  # PAD : 2
        for idx, s in enumerate(test_x):
            length = len(s)
            if length < MAXLEN:
                test_x_pad[idx, :length] = np.array(s)
            else:
                test_x_pad[idx:, :MAXLEN] = np.array(s)[:MAXLEN]  # Truncate from the front
            
        # feed_dict 
        # feed_dict = {}
#         feed_dict = {input_x: test_x_pad}
        feed_dict = {input_x: test_x_pad, input_len: test_x_len}
        p_1, p_2 = sess.run([pred_1, pred_2], feed_dict=feed_dict)
        
        # squared error
        se_1 += np.sum(np.square(p_1 - test_y))
        se_2 += np.sum(np.square(p_2 - test_y))
        total_1 += len(p_1)
        total_2 += len(p_2)

    # root mean squared error
    test_rmse_1 = np.sqrt(se_1 / total_1)
    test_rmse_2 = np.sqrt(se_2 / total_2)
    print('Test RMSE 1 = %.4f, RMSE 2 = %.4f' % (test_rmse_1, test_rmse_2))
    
    if best_rmse > test_rmse_1:
        best_rmse = test_rmse_1
        best_epoch = epoch
        
    if best_rmse > test_rmse_2:
        best_rmse = test_rmse_2
        best_epoch = epoch

[ 50/157] loss1 = 4.2347, time elapsed = 0.27
[ 50/157] loss2 = 4.1360, time elapsed = 0.27
[100/157] loss1 = 4.7592, time elapsed = 0.28
[100/157] loss2 = 4.4679, time elapsed = 0.28
[150/157] loss1 = 3.7254, time elapsed = 0.27
[150/157] loss2 = 2.3984, time elapsed = 0.27
Epoch   1 >> Epoch loss1: 752.4133 , time elapsed 61.0098
Epoch   1 >> Epoch loss2: 691.3803 , time elapsed 61.0098
Test RMSE 1 = 6.2188, RMSE 2 = 5.7329
[ 50/157] loss1 = 2.5854, time elapsed = 0.27
[ 50/157] loss2 = 0.7119, time elapsed = 0.27
[100/157] loss1 = 3.9304, time elapsed = 0.27
[100/157] loss2 = 1.4525, time elapsed = 0.27
[150/157] loss1 = 2.3036, time elapsed = 0.28
[150/157] loss2 = 0.8430, time elapsed = 0.28
Epoch   2 >> Epoch loss1: 500.6225 , time elapsed 60.4023
Epoch   2 >> Epoch loss2: 197.2066 , time elapsed 60.4023
Test RMSE 1 = 5.8452, RMSE 2 = 4.9640
[ 50/157] loss1 = 3.1272, time elapsed = 0.28
[ 50/157] loss2 = 1.4280, time elapsed = 0.28
[100/157] loss1 = 2.3763, time elapsed = 0.27
[1

KeyboardInterrupt: ignored

In [26]:
print('Best RMSE is %.4f at EPOCH %d' % (best_rmse, best_epoch))

Best RMSE is 4.6385 at EPOCH 79


In [0]:
!nvidia-smi

Thu Jul 25 02:35:35 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    24W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  